Бабенко Р. Лабораторная работа номер 3.

Задание 1. Изменить функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [62]:
import numpy as np

def calc_logloss(y, y_pred):
    dim = y.shape[0]
    err = 0
    for i in range(dim):
        if y_pred[i] != 1.0 and y_pred[i] != 0:
            err -= y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    rez = err / dim
    return rez

Используем цикл чтобы расспатривать вес каждого признака отдельно, и условием не рассматриваем те признаки, в которых y_pred равен нулю или единице(чтобы нули не попалали в логарифм). 

In [63]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
print(calc_logloss(y1, y_pred1))

0.11157177565710485


Тестовый пример, на котором выдавало исключение

Задание 2. Подобрать аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [64]:
import numpy as np
import matplotlib.pyplot as plt

X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

W = eval_model(X_st, y, iterations=10000, eta=1)

0 [ 0.11732727 -1.58914029  0.62030812  0.95688359] 1.1785958344356262
1000 [-10.79956087  -1.38842301  -2.43773192   9.1781253 ] 0.23165947539015325
2000 [-15.46737926  -1.78489608  -3.84058484  12.94996634] 0.19317640063439964
3000 [-19.06453024  -2.10307143  -4.89402886  15.87355904] 0.1703820922419324
4000 [-22.02639821  -2.36864314  -5.74972113  18.28151701] 0.15496509524744978
5000 [-24.55062127  -2.59641317  -6.47260633  20.33229852] 0.1437907257559476
6000 [-26.75533033  -2.7959032   -7.1002527   22.12170973] 0.13528147157282963
7000 [-28.71761855  -2.97355287  -7.65663776  23.71251207] 0.12855136478278142
8000 [-30.49042692  -3.13388494  -8.15795259  25.14787759] 0.12306625393611917
9000 [-32.11152175  -3.28017801  -8.61559398  26.45863223] 0.11848606975828083


Не совсем понял, но при увеличении количестви итераций и увеличивая шаг, значение log loss постоянно уменьшалось... Задал значения как 10000 и 1.

Задание 3.

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [65]:
def calc_pred_proba(W, X): 
    y_pred_proba = np.squeeze(sigmoid(np.dot(X, W)))
    return y_pred_proba

In [66]:
calc_pred_proba(W, X_st) # вероятности принадлежности к классу 1.

array([3.76907782e-01, 1.43398442e-02, 9.99999995e-01, 1.09706210e-07,
       9.32650715e-01, 6.42051408e-02, 1.00000000e+00, 2.06413788e-02,
       6.05682604e-01, 9.99999459e-01])

Вероятности принадлежности к классу 1

Задание 3. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred)

In [67]:
def calc_pred(W, X_st):
    y_pred = []
    for i in calc_pred_proba(W, X_st):
        if i > 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

За пороговую вероятность принадлезности к классу 1 возмём вероятность большую 0.5, и циклом проходясь по каждой полученной йз calc_pred_proba вероятности, смотрим, принадлежит ли объект к классу 0 или 1 

In [68]:
calc_pred(W, X_st)

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]

Для сравнения с настояшеё принадлежности к определённому классу рассотрим узначальный массив y нашей модели. Результат - предикт работает достаточно точно.

In [69]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

Задание 5. Реазовать функции:

Accuracy

Матрицы ошибок

Точности и полноты

F1 score

In [103]:
def Accuracy(W, X):
    m = X.shape[0]
    count = 0
    pred = calc_pred(W, X) # предикты, к какому классу относиться объект
    for i in range(m):
        if pred[i] == y[i]: # если предикт совпал с ответом
            count += 1 # увеличиваем счётчик
    rez = count / m # отношение правильных ответов к общему количеству
    return rez

In [111]:
def error_matrix(W, X):
    m = X.shape[0]
    pred = calc_pred(W, X)
    TP = 0 # истинные положительное срабатывания
    FP = 0 # ложные положительное срабатывания
    FN = 0 # ложные отрицательное срабатывания
    TN = 0 # Истинные отрицательное срабатывания
    for i in range(m):
        if pred[i] == 1 and y[i] == 1:
            TP += 1
        elif pred[i] == 1 and y[i] == 0:
            FP += 1
        elif pred[i] == 0 and y[i] == 1:
            FN += 1
        else:
            TN += 1
    return [[TP, FP], [FN, TN]] # возращаем матрицу

In [105]:
def precision(W, X): # вычислние точности
    m = X.shape[0]
    pred = calc_pred(W, X)
    TP = 0 # счётчик истинных позитивных срабатываний
    FP = 0 # счётчик ложных похитивных срабатываний
    for i in range(m):
        if pred[i] == 1 and y[i] == 1: # если срабатывание истинное позитивное 
            TP += 1
        elif pred[i] == 1 and y[i] == 0: # если срабртывание ложное позитивное
            FP += 1
    rez = TP / TP + FP
    return rez

In [106]:
def recall(W, X): # вычисление полноты
    m = X.shape[0]
    pred = calc_pred(W, X)
    TP = 0 # счётчик истинных позитивных срабатываний
    FN = 0 # счётчик ложных негатиных срабатываний
    for i in range(m):
        if pred[i] == 1 and y[i] == 1: # если срабатывание истинное позитивное 
            TP += 1
        elif pred[i] == 0 and y[i] == 1: # если срабртывание ложное позитивное
            FP += 1
    rez = TP / TP + FN
    return rez

In [101]:
def F_measure(W, X, b): # вычисление f - меры
    prec = precision(W, X)
    rec = recall(W, X)

    # формула среднего гармонического между точностью и полнотой
    rez = (1 + b**2) * (prec * rec) / (b**2 * prec * rec) 

    return rez